# Tuning a Convolutional Network

# Setup

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
torchvision.disable_beta_transforms_warning()
import torchvision.transforms.v2 as transforms

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
%config InlineRenderer.figure_format = 'retina'

import torch.utils.tensorboard as tb

log_dir = 'logs'

In [5]:
transform = transforms.Compose([
    transforms.ToImageTensor(),
    transforms.ConvertImageDtype()
])

cifar = torchvision.datasets.CIFAR10("../../data/torch/cifar", download=True, transform=transform)
train_size = int(0.8 * len(cifar))
train_data, valid_data = torch.utils.data.random_split(cifar, [train_size, len(cifar) - train_size])

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(len(cifar))

Files already downloaded and verified
50000


In [6]:
mean = []
for x, _ in cifar:
    mean.append(torch.mean(x, dim=(1, 2)))
mean = torch.stack(mean, dim=0).mean(dim=0)
std = []
for x, _ in cifar:
    std.append(((x - mean[:,np.newaxis,np.newaxis]) ** 2).mean(dim=(1, 2)))
std = torch.stack(std, dim=0).mean(dim=0).sqrt()
print(mean, std)

tensor([0.4914, 0.4822, 0.4465]) tensor([0.2470, 0.2435, 0.2616])


In [7]:
cifar_mean = (0.4914, 0.4822, 0.4465)
cifar_std = (0.2470, 0.2435, 0.2616)

normalize = transforms.Normalize(cifar_mean, cifar_std)

if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = 'cpu'
print(device)

mps


# CNN Model

